In [1]:
import h5py

In [3]:
with h5py.File('../mvtec_dataset.h5', 'r') as f:
    def print_name(name):
        print(name, f[name].shape if hasattr(f[name], 'shape') else '')
    f.visit(print_name)

bottle 
bottle/test 
bottle/test/broken_large 
bottle/test/broken_large/images (20, 128, 128, 3)
bottle/test/broken_small 
bottle/test/broken_small/images (22, 128, 128, 3)
bottle/test/contamination 
bottle/test/contamination/images (21, 128, 128, 3)
bottle/test/good 
bottle/test/good/images (20, 128, 128, 3)
bottle/train 
bottle/train/good 
bottle/train/good/images (209, 128, 128, 3)
cable 
cable/test 
cable/test/bent_wire 
cable/test/bent_wire/images (13, 128, 128, 3)
cable/test/cable_swap 
cable/test/cable_swap/images (12, 128, 128, 3)
cable/test/combined 
cable/test/combined/images (11, 128, 128, 3)
cable/test/cut_inner_insulation 
cable/test/cut_inner_insulation/images (14, 128, 128, 3)
cable/test/cut_outer_insulation 
cable/test/cut_outer_insulation/images (10, 128, 128, 3)
cable/test/good 
cable/test/good/images (58, 128, 128, 3)
cable/test/missing_cable 
cable/test/missing_cable/images (12, 128, 128, 3)
cable/test/missing_wire 
cable/test/missing_wire/images (10, 128, 128, 3)
c